In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
path =  "../data/"
dataset = "cb12/"

raw_path = path + dataset + "raw/" 
interim_path = path + dataset + "interim/"
processed_path = path + dataset + "processed/"

# Step 1: Load data

### User data

In [3]:
df_user = pd.read_csv(raw_path + 'users.tsv', header=0, sep='\t')
print(df_user.shape)
df_user.dropna(axis=0, how='any', thresh=None, subset=['UserID', 'City', 'State', 'Country', 'DegreeType', 'Major', 'WorkHistoryCount', 'TotalYearsExperience'], inplace=True)
print(df_user.shape)

(389708, 15)
(286569, 15)


### Application records

In [4]:
df_app = pd.read_csv(raw_path + 'applications.csv', header=0, sep='\t')
print(df_app.shape)

(1603111, 6)


In [5]:
print("Number of unique Jobs: ", len(df_app.JobID.unique()))
print("Number of unique Users: ", len(df_app.UserID.unique()))
print(df_app.groupby('UserID').size().describe())

Number of unique Jobs:  365668
Number of unique Users:  321235
count    321235.000000
mean          4.990462
std          11.418487
min           1.000000
25%           1.000000
50%           2.000000
75%           5.000000
max        2473.000000
dtype: float64


### Keep applications only for valid users
Valid user: have all infor

In [6]:
df_app_valid = df_app[df_app.UserID.isin(df_user.UserID)]
print(df_app_valid.shape)
print("Number of unique Jobs: ", len(df_app_valid.JobID.unique()))
print("Number of unique Users: ", len(df_app_valid.UserID.unique()))
# print(df_app_valid.groupby('UserID').size().describe())

(1187652, 6)
Number of unique Jobs:  321417
Number of unique Users:  236343


In [7]:
print("Start Time: {}".format(pd.to_datetime(df_app_valid["ApplicationDate"].min(), unit="s")))
print("Start Time: {}".format(pd.to_datetime(df_app_valid["ApplicationDate"].max(), unit="s")))

# remove NaN values
df_app_valid = df_app_valid[np.isfinite(df_app_valid['ApplicationDate'])]
# convert back to long from float
df_app_valid['ApplicationDate'] = df_app_valid['ApplicationDate'].astype(np.int64)

Start Time: 2012-04-01 00:00:21
Start Time: 2012-06-26 23:59:50


### Job records

In [ ]:
df_job = pd.read_csv(raw_path + "jobs.tsv", header=0, sep='\t', on_bad_lines='skip')
print(df_job.shape)
df_job['StartDate'] = df_job['StartDate'].astype("datetime64[ms]").astype(np.int64) // 10**9
df_job['EndDate'] = df_job['EndDate'].astype("datetime64[ms]").astype(np.int64) // 10**9

### Drop incomplete rows for comparison purpose

In [ ]:
# df_job.dropna(axis=0, how='any', thresh=None, subset=['JobID', 'Title', 'Description', 'Requirements', 'City', 'State', 'Country'], inplace=True)
# print(df_job.shape)

In [ ]:
# df_job['Requirements'] = df_job['Requirements'].map(lambda x:re.sub('<[^<]+?>', '', x)).map(lambda x:re.sub('\\\\r', '', x)).map(lambda x:re.sub('\\\\n', '', x)).map(lambda x:re.sub('&nbsp;', ' ', x)).map(lambda x:re.sub('[—]+', ' ', x)).map(lambda x:re.sub('/', ' ', x))
# df_job['Description'] = df_job['Description'].map(lambda x:re.sub('<[^<]+?>', '', x)).map(lambda x:re.sub('\\\\r', '', x)).map(lambda x:re.sub('\\\\n', '', x)).map(lambda x:re.sub('&nbsp;', ' ', x)).map(lambda x:re.sub('[—]+', ' ', x)).map(lambda x:re.sub('/', ' ', x))
# df_job['Title'] = df_job['Title'].map(lambda x:re.sub('/', ' ', x))

# df_job['Requirements'] = df_job['Requirements'].str.lower()
# df_job['Description'] = df_job['Description'].str.lower()
# df_job['Title'] = df_job['Title'].str.lower()
# df_job.to_csv(interim_path + "jobs_cleaned.tsv", sep='\t')

### Or load directly

In [10]:
df_job = pd.read_csv(interim_path + "jobs_cleaned.tsv", sep='\t')
print("Number of unique Jobs:", len(df_job.JobID.unique()))
print("Number of unique Title", len(df_job.Title.unique()))
print("Number of unique Description:", len(df_job.Description.unique()))
print("Number of unique Requirements", len(df_job.Requirements.unique()))
print("Number of unique Cities: " + str(len(df_job.City.unique())))
#print(jobs['city'].value_counts(normalize=True) * 100)
print("Number of unique States: " + str(len(df_job.State.unique())))
#print(jobs['state'].value_counts(normalize=True) * 100)
print("Number of unique countries: " + str(len(df_job.Country.unique())))
#print(jobs['country'].value_counts(normalize=True) * 100)
print("Number of unique zip codes: " + str(len(df_job.Zip5.unique())))

/usr/local/anaconda3/envs/py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Number of unique Jobs: 1050503
Number of unique Title 413201
Number of unique Description: 718456
Number of unique Requirements 513490
Number of unique Cities: 10995
Number of unique States: 60
Number of unique countries: 65
Number of unique zip codes: 42681


### Add job information to df_app

In [11]:
df_app_new = pd.merge(df_app_valid, df_job, on='JobID', how='inner')
df_app_new.sort_values(by=['UserID'], inplace=True)
df_app_new.drop(columns='WindowID_y',inplace=True)
df_app_new.rename(columns = {'WindowID_x': 'WindowID'}, inplace = True)
print(df_app_new.shape)

(1171649, 16)


In [12]:
print("Number of unique Jobs: ", len(df_app_new.JobID.unique()))
print("Number of unique Users: ", len(df_app_new.UserID.unique()))
print(df_app_new.groupby('UserID').size().describe())

Number of unique Jobs:  313457
Number of unique Users:  234818
count    234818.000000
mean          4.989605
std          11.911864
min           1.000000
25%           1.000000
50%           2.000000
75%           5.000000
max        2445.000000
dtype: float64


# Step 2: Make sessions

In [13]:
def make_sessions(data, session_threshold = 30*60, is_ordered = False, user_key = 'UserID', job_key = 'JobID', time_key = 'ApplicationDate'):
    # Assigns sessionn ids to the events in data without grouping keys
    if not is_ordered:
        # Sort data by user and time
        data.sort_values(by=[user_key, time_key], ascending=True, inplace=True)
        
        # Compute the time difference between queriest
        time_diff = np.diff(data[time_key].values)
        
        # Check which of them are bigger than session_threshold
        split_session = time_diff > session_threshold
        split_session = np.r_[True, split_session]
        
        # Check when the user changes is data
        new_user = data['UserID'].values[1:] != data['UserID'].values[:-1]
        new_user = np.r_[True, new_user]
        
        # A new sessions stars when at least one of the two conditions is verified
        new_session = np.logical_or(new_user, split_session)
        
        # Compute the session ids
        session_ids = np.cumsum(new_session)
        data['SessionID'] = session_ids
        return data  

### 30mins

In [ ]:
df_sessions_30 = make_sessions(df_app_new, session_threshold=30 * 60, time_key='ApplicationDate', is_ordered=False)
print(df_sessions_30.shape)
print(df_sessions_30.head(10))

# Drop duplicate interactions
df_sessions_30 = df_sessions_30.drop_duplicates(['SessionID','ApplicationDate'])
print(df_sessions_30.shape)

print('Original data:')
print('Num jobs: {}'.format(df_sessions_30.JobID.nunique()))
print('Num users: {}'.format(df_sessions_30.UserID.nunique()))
print('Num sessions: {}'.format(df_sessions_30.SessionID.nunique()))

In [15]:
print('Filtering data')
# Drop duplicate applications within the same session
df_sessions_30.drop_duplicates(subset=['JobID', 'UserID'], keep='first', inplace=True)
print(df_sessions_30.shape)

# Keep jobs with >=1 applications
job_pop = df_sessions_30.JobID.value_counts()
#good_items = item_pop[item_pop >= 5].index
good_jobs = job_pop[job_pop >= 1].index
print('Number of jobs with >= 1 applications: ', len(good_jobs))
df_session_dense_30 = df_sessions_30[df_sessions_30.JobID.isin(good_jobs)]
print(df_session_dense_30.shape)


# Remove sessions with length < 2
session_length = df_session_dense_30.SessionID.value_counts()
good_sessions = session_length[session_length >= 2].index
print('Number of sessions with >= 2 applications: ', len(good_sessions))
df_session_dense_30 = df_session_dense_30[df_session_dense_30.SessionID.isin(good_sessions)]
print(df_session_dense_30.shape)

# Keep only returning users (with >= 1 sessions) and remove overly active ones (>=200 sessions)
sess_per_user = df_session_dense_30.groupby('UserID')['SessionID'].nunique()
good_users = sess_per_user[(sess_per_user >= 1) & (sess_per_user < 200000)].index
df_session_dense_30 = df_session_dense_30[df_session_dense_30.UserID.isin(good_users)]
print('Filtered data:')
print('Num jobs: {}'.format(df_session_dense_30.JobID.nunique()))
print('Num users: {}'.format(df_session_dense_30.UserID.nunique()))
print('Num sessions: {}'.format(df_session_dense_30.SessionID.nunique()))
print(df_session_dense_30.shape)

# store_path = "../data/cb12/"
# df_session_dense_30.to_csv(store_path + "session_filtered_30_consider_user.csv", sep='\t')
df_session_dense_30.to_csv(interim_path + "session_filtered_30_consider_user.csv", sep='\t')

Filtering data


/usr/local/anaconda3/envs/py3.7/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(968885, 17)
Number of jobs with >= 1 applications:  301901
(968885, 17)
Number of sessions with >= 2 applications:  173627
(676814, 17)
Filtered data:
Num jobs: 225268
Num users: 117301
Num sessions: 173627
(676814, 17)


# Step 3. Create train and test set 
a time-based (14 days) split

In [16]:
def last_n_days_out_split(data, n=1, user_key = 'UserID', job_key = 'JobID', session_key = 'SessionID', time_key = 'ApplicationDate', clean_test=True, min_session_length=2):
    """
    Assign the sessions in the last n days to the test set and remaining to the training one
    """
    DAY = 24 * 60 * 60
    data.sort_values(by=[user_key, time_key], inplace=True)
    end_time = data[time_key].max()
    print('end_time', end_time)
    test_start = end_time - n * DAY
    print('test_start', test_start)
    
    # get train and test indicies
    session_max_times = data.groupby(session_key)[time_key].max()
    session_train = session_max_times[session_max_times < test_start].index
    session_test = session_max_times[session_max_times >= test_start].index
    
    
    # in1d: Returns a boolean array the same length as ar1 that is True where an element of ar1 is in ar2 and False otherwise.
    train = data[np.in1d(data[session_key], session_train)].copy()
    test = data[np.in1d(data[session_key], session_test)].copy()
    
    if clean_test:
        before_jobs = len(test[job_key].unique())
        # Remove jobs which do not occur in the train set
        test = test[np.in1d(test[job_key],train[job_key])]
        after_jobs = len(test[job_key].unique())
        print("Before job count: " + str(before_jobs))
        print("After job count: " + str(after_jobs))
        
        
        # Remove sessions in test shorter than min_session_length
        tslength = test.groupby(session_key).size()
        test = test[np.in1d(test[session_key],tslength[tslength >= min_session_length].index)].copy()
    

    return train, test

In [17]:
train_full_sessions_30, test_sessions_30 = last_n_days_out_split(df_session_dense_30, n=14, user_key = 'UserID', job_key = 'JobID', session_key = 'SessionID', time_key = 'ApplicationDate', clean_test=True)
train_valid_sessions_30, valid_sessions_30 = last_n_days_out_split(train_full_sessions_30, n=14, user_key = 'UserID', job_key = 'JobID', session_key = 'SessionID', time_key = 'ApplicationDate', clean_test=True)

end_time 1340755190
test_start 1339545590
Before job count: 31355
After job count: 14059
end_time 1339545569
test_start 1338335969
Before job count: 45802
After job count: 17298


In [18]:
print(train_full_sessions_30.shape)
print(test_sessions_30.shape)
print(train_valid_sessions_30.shape)
print(valid_sessions_30.shape)

(586434, 17)
(52035, 17)
(465084, 17)
(54676, 17)


In [19]:
# print statistics
train_job_len = len(train_full_sessions_30.JobID.unique())
print("Number of Training Jobs: " + str(train_job_len))
train_user_len = len(train_full_sessions_30.UserID.unique())
print("Number of Training Users: " + str(train_user_len))
train_len = len(train_full_sessions_30.SessionID.unique())
print("Number of Training Sessions: " + str(train_len))
print()

test_job_len = len(test_sessions_30.JobID.unique())
print("Number of Testing Jobs: " + str(test_job_len))
test_user_len = len(test_sessions_30.UserID.unique())
print("Number of Testing Users: " + str(test_user_len))
test_len = len(test_sessions_30.SessionID.unique())
print("Number of Testing Sessions: " + str(test_len))

Number of Training Jobs: 207972
Number of Training Users: 104583
Number of Training Sessions: 153268

Number of Testing Jobs: 13166
Number of Testing Users: 10031
Number of Testing Sessions: 11759


In [20]:
merged_jobs_30 = train_full_sessions_30.append(test_sessions_30, ignore_index=True)
print(merged_jobs_30.shape)
merged_job_len = len(merged_jobs_30.JobID.unique())
print("Number of Train + Test Jobs: " + str(merged_job_len))
print('\n')
merged_user_len = len(merged_jobs_30.UserID.unique())
print("Number of Train + Test Users: " + str(merged_user_len))
print('\n')

print("Number of Train + Test Sessions: " + str(train_len + test_len))
print('\n')

print("Number of Train Validating Sessions: " + str(len(train_valid_sessions_30.SessionID.unique())))
print("Number of Test Validating Sessions: " + str(len(valid_sessions_30.SessionID.unique())))

(638469, 17)
Number of Train + Test Jobs: 207972


Number of Train + Test Users: 111785


Number of Train + Test Sessions: 165027


Number of Train Validating Sessions: 127412
Number of Test Validating Sessions: 12343


In [21]:
train_full_sessions_30.to_csv(processed_path + "train_14d_30_consider_user.csv", sep='\t')
test_sessions_30.to_csv(processed_path + "test_14d_30_consider_user.csv", sep='\t')
train_valid_sessions_30.to_csv(processed_path + "valid_train_14d_30_consider_user.csv", sep='\t')
valid_sessions_30.to_csv(processed_path + "valid_test_14d_30_consider_user.csv", sep='\t')